In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-privacy

In [ ]:
import tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow_privacy.privacy.dp_query.normalized_query import NormalizedQuery
from tensorflow_privacy.privacy.dp_query.no_privacy_query import NoPrivacyAverageQuery
from tensorflow_privacy.privacy.membership_inference_attack import membership_inference_attack as mia
import numpy as np
import os
from PIL import Image

batch_size = 32
img_height = 75
img_width = 75


# Set the seed for reproducibility
tf.random.set_seed(123)

# Define the paths to the data and the model checkpoints
# data_dir = '/content/drive/MyDrive/MTP-2/chest_xray'
model_dir = '/content/drive/MyDrive/MTP-2'

# Define the privacy parameters
# epsilon = 1.0
# delta = 1e-6

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/MTP-2/chest_xray/train',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/MTP-2/chest_xray/test',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

class_names = train_ds.class_names
num_classes = len(class_names)

# Extract the images and labels from the datasets
train_images, train_labels = [], []
test_images, test_labels = [], []
for images, labels in train_ds:
    train_images.append(images.numpy())
    train_labels.append(labels.numpy())
train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)
for images, labels in test_ds:
    test_images.append(images.numpy())
    test_labels.append(labels.numpy())
test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)

In [ ]:

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.metrics import precision_score, recall_score, f1_score


base_model = DenseNet121(include_top=False, weights=None, input_shape=(img_height, img_width, 3)) #32

# base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3)) #32
# base_model = keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3)) #75
# base_model = keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3)) #75
# weights_path = 'https://github.com/zoogzog/chexnet/raw/master/model.h5'
# base_model.load_weights(weights_path)
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# # x = Dropout(0.5)(x) # Regularization
predictions = Dense(num_classes, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)
# model.layers.po

epoch=2
model.compile(optimizer=Adam(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



# Train the model without differential privacy
model.fit(train_ds, validation_data=val_ds, epochs=epoch)



test_data = val_ds.take(1)
test_images, test_labels = next(iter(test_data))
output = model(test_images)
predictions = tf.argmax(output, axis=1)
test_labels = np.concatenate([y for x, y in test_data], axis=0)

# # Calculate accuracy
# accuracy = np.mean(np.equal(predictions, test_labels))
# print("Accuracy without differential privacy: {:.2f}%".format(accuracy * 100))
# test_labels = np.concatenate([y for x, y in test_data], axis=0)
precision = precision_score(test_labels, predictions, average='weighted')
recall = recall_score(test_labels, predictions, average='weighted')
f1 = f1_score(test_labels, predictions, average='weighted')
print("Precision without differential privacy: {:.2f}%".format(precision * 100))
print("Recall without differential privacy: {:.2f}%".format(recall * 100))
print("F1 without differential privacy: {:.2f}%".format(f1 * 100))


In [ ]:
from tensorflow_privacy.privacy.dp_query.gaussian_query import GaussianSumQuery
from tensorflow_privacy.privacy.dp_query.normalized_query import NormalizedQuery
from tensorflow_privacy.privacy.dp_query.no_privacy_query import NoPrivacyAverageQuery

# Set the privacy parameters
epsilon = 1.0
delta = 1e-6

# Define the differential privacy mechanism with Gaussian noise
gaussian_query = GaussianSumQuery(epsilon, delta)
normalized_query = NormalizedQuery(gaussian_query, denominator=1.0)
noisy_avg = NoPrivacyAverageQuery()
# print(gaussian_query.shape,"gaussian query")
# Clone the model and add differential privacy to the weights
model_gau = keras.models.clone_model(model)
# for layer in base_model_gau.layers:
#     layer.trainable = False
model_gau.set_weights(model.get_weights())
# global_state = gaussian_query.initial_global_state( )
global_state = normalized_query.initial_global_state()


for layer in model_gau.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
        # layer.kernel = normalized_query(layer.kernel)
        # layer.kernel = normalized_query.get_noised_result(layer.kernel, global_state)
        # layer.kernel = tf.reshape(normalized_query.get_noised_result(tf.reshape(layer.kernel, (-1, layer.kernel.shape[-1])), global_state), layer.kernel.shape)
        # layer.kernel = tf.reshape(layer.kernel, (-1, layer.kernel.shape[-1]))
        # layer.kernel = normalized_query.get_noised_result(layer.kernel, global_state)
        # layer.kernel = tf.reshape(layer.kernel, layer.kernel.shape.as_list()[:-1] + [-1])
        # layer.kernel = normalized_query.get_noised_result(tf.reshape(layer.kernel, (-1, layer.kernel.shape[-1])), global_state)
        kernel, _ ,_= normalized_query.get_noised_result(tf.reshape(layer.kernel, (-1, layer.kernel.shape[-1])), global_state)

        layer.kernel = tf.reshape(kernel, layer.kernel.shape)

        # layer.kernel = tf.reshape(layer.kernel, list(layer.kernel.shape))








# Compile the model with a loss and metrics
model_gau.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with the differentially private weights
history = model_gau.fit(train_ds, validation_data=val_ds, epochs=1)

# Evaluate the model's accuracy with differential privacy
test_data = val_ds.take(1)
test_images, test_labels = next(iter(test_data))
output = model_gau(test_images)
# output = model(test_images)
predictions_g = tf.argmax(output, axis=1)
test_labels = np.concatenate([y for x, y in test_data], axis=0)

# Calculate accuracy
# accuracy = np.mean(np.equal(predictions, test_labels))
# print("Accuracy with differential privacy (Gaussian noise): {:.2f}%".format(accuracy * 100))
precision_g = precision_score(test_labels, predictions_g, average='weighted')
recall_g = recall_score(test_labels, predictions_g, average='weighted')
f1_g = f1_score(test_labels, predictions_g, average='weighted')
print("Precision with differential privacy (Gaussian noise): {:.2f}%".format(precision_g * 100))
print("Recall with differential privacy (Gaussian noise): {:.2f}%".format(recall_g * 100))
print("F1 score with differential privacy (Gaussian noise): {:.2f}%".format(f1_g * 100))



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Set the privacy parameters
epsilon = 0.25
delta = 1e-6

# Define the differential privacy mechanism with Laplacian noise
def add_noise(weights, epsilon):
    # print("weight shape", weights.shape)
    noise = np.random.laplace(scale=2.0 / epsilon, size=weights.shape)
    return weights + noise

# Clone the model and add differential privacy to the weights
model_lap = keras.models.clone_model(model)
model_lap.set_weights(model.get_weights())
for layer in model_lap.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
        layer.kernel = add_noise(layer.kernel, epsilon)

# Compile the model with a loss and metrics
model_lap.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with the differentially private weights
history = model_lap.fit(train_ds, validation_data=val_ds, epochs=1)



# Evaluate the model's accuracy with differential privacy
# Evaluate the model's accuracy with differential privacy
test_data = val_ds.take(1)
output = model_lap.predict(test_data) # use predict() instead of model_dp()
predictions_l = tf.argmax(output, axis=1)
test_labels = np.concatenate([y for x, y in test_data], axis=0)
# accuracy = tf.reduce_mean(tf.cast(predictions == test_labels, tf.float32))
# print("Accuracy with differential privacy (Laplacian noise): {:.2f}%".format((accuracy * 100)))
precision_l = precision_score(test_labels, predictions_l, average='weighted')
recall_l = recall_score(test_labels, predictions_l, average='weighted')
f1_l = f1_score(test_labels, predictions_l, average='weighted')
print("Precision with differential privacy (Laplacian noise): {:.2f}%".format(precision_l * 100))
print("Recall with differential privacy (Laplacian noise): {:.2f}%".format(recall_l * 100))
print("F1 score with differential privacy (Laplacian noise): {:.2f}%".format(f1_l * 100))



In [ ]:

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow_privacy.privacy.dp_query.gaussian_query import GaussianSumQuery

# Set the privacy parameters
epsilon = 0.25
delta = 1e-6
laplacian_scale = 2.0 / epsilon

# Define the differential privacy mechanism with composite noise
gaussian_query = GaussianSumQuery(epsilon, delta)
normalized_query = NormalizedQuery(gaussian_query, denominator=1.0)
noisy_avg = NoPrivacyAverageQuery()
global_state = normalized_query.initial_global_state()

# Clone the model and add differential privacy to the weights
model_com = keras.models.clone_model(model)
model_com.set_weights(model.get_weights())

for layer in model_com.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
        # Add composite noise to the weights
        noise, _, _ = normalized_query.get_noised_result(tf.reshape(layer.kernel, (-1, layer.kernel.shape[-1])), global_state)
        noise = tf.reshape(noise, layer.kernel.shape)
        laplacian_noise = np.random.laplace(scale=laplacian_scale, size=layer.kernel.shape)
        layer.kernel = layer.kernel + noise + laplacian_noise

# Compile the model with a loss and metrics
model_com.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with the differentially private weights
history = model_com.fit(train_ds, validation_data=val_ds, epochs=1)

# Evaluate the model's accuracy with differential privacy
test_data = val_ds.take(1)
output = model_com.predict(test_data) # use predict() instead of model_dp()
predictions_c = tf.argmax(output, axis=1)
test_labels = np.concatenate([y for x, y in test_data], axis=0)
accuracy = tf.reduce_mean(tf.cast(predictions_c == test_labels, tf.float32))
print("Accuracy with differential privacy (Composite noise): {:.2f}%".format(accuracy * 100))


In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras import layers

# # Generate synthetic data

# num_features = 10
# x_train = np.random.rand(num_samples, num_features)
# y_train = np.random.randint(2, size=num_samples)

# # Train a machine learning model
# model = keras.Sequential([
#     layers.Dense(64, activation='relu', input_shape=(num_features,)),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(1, activation='sigmoid')
# ])
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])
# model.fit(x_train, y_train, epochs=10)

# # Split the data into train and test sets
# x_test = np.random.rand(num_samples, num_features)
# y_test = np.random.randint(2, size=num_samples)

# Perform a membership inference attack
train_preds = model.predict(train_images)
test_preds = model.predict(test_images)
train_labels = np.ones_like(train_labels)
test_labels = np.zeros_like(test_labels)
train_data = np.concatenate((train_preds, train_labels.reshape(-1, 1)), axis=1)
test_data = np.concatenate((test_preds, test_labels.reshape(-1, 1)), axis=1)
num_samples=len(train_data)+len(test_data)
# Train a binary classifier to distinguish between train and test data
attack_model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(3,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])



num_samples = len(train_data) + len(test_data)
attack_targets = np.concatenate((np.ones(len(train_data)), np.zeros(len(test_data))))
# attack_targets = np.concatenate((attack_targets, np.zeros(len(train_data))))
# attack_targets = np.concatenate((attack_targets, np.ones(len(test_data))))

attack_model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

attack_model.fit(np.concatenate((train_data, test_data)), attack_targets, epochs=2, batch_size=32)

# attack_model.fit(np.concatenate((train_data, test_data)), np.concatenate((np.ones(num_samples), np.zeros(num_samples))),
#                  epochs=1, batch_size=32)

# Evaluate the attack model
train_attack_preds = attack_model.predict(train_data)
test_attack_preds = attack_model.predict(test_data)
train_attack_acc = np.sum(train_attack_preds > 0.5) / num_samples
test_attack_acc = np.sum(test_attack_preds <= 0.5) / num_samples

print("Membership inference attack accuracy on train data: ", train_attack_acc)
print("Membership inference attack accuracy on test data: ", test_attack_acc)

1/1 [==============================] - 0s 452ms/step
Epoch 1/2
36/36 [==============================] - 1s 3ms/step - loss: 0.6055 - accuracy: 0.7835
Epoch 2/2
1/1 [==============================] - 0s 22ms/step
Membership inference attack accuracy on train data:  0.9721739130434782
Membership inference attack accuracy on test data:  0.0


#Commented Code